# Example of the SASRec training and inference stages
Note that all the given examples can be run without using PySpark, using only Pandas

In [1]:
import lightning as L
from lightning.pytorch.loggers import CSVLogger
from lightning.pytorch.callbacks import ModelCheckpoint
from torch.utils.data import DataLoader
import torch

from replay.metrics import OfflineMetrics, Recall, Precision, MAP, NDCG, HitRate, MRR
from replay.metrics.torch_metrics_builder import metrics_to_df
from replay.splitters import LastNSplitter
from replay.data import (
    FeatureHint,
    FeatureInfo,
    FeatureSchema,
    FeatureSource,
    FeatureType,
    Dataset,
)
from replay.models.nn.optimizer_utils import FatOptimizerFactory
from replay.models.nn.sequential.callbacks import (
    ValidationMetricsCallback,
    SparkPredictionCallback,
    PandasPredictionCallback,
    TorchPredictionCallback,
    QueryEmbeddingsPredictionCallback,
)
from replay.models.nn.sequential.postprocessors import RemoveSeenItems
from replay.data.nn import SequenceTokenizer, SequentialDataset, TensorFeatureSource, TensorSchema, TensorFeatureInfo
from replay.models.nn.sequential import SasRec
from replay.models.nn.sequential.sasrec import (
    SasRecPredictionDataset,
    SasRecTrainingDataset,
    SasRecValidationDataset,
    SasRecPredictionBatch,
    SasRecModel,
)
import pandas as pd

## Prepare data
### Load raw movielens-1M interactions, item features and user features.
In the current implementation, the SASRec does not take into account the features of items or users. They are only used to get a complete list of users and items.

In [ ]:
!pip install rs-datasets

In [2]:
from rs_datasets import MovieLens

In [3]:
movielens = MovieLens("1m")
interactions = movielens.ratings
user_features = movielens.users
item_features = movielens.items

In [4]:
interactions.head()

,user_id,item_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [5]:
user_features.head()

,user_id,gender,age,occupation,zip_code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [6]:
item_features.head()

,item_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


Removing duplicates in the timestamp column without changing the original items order where timestamp is the same

In [7]:
interactions["timestamp"] = interactions["timestamp"].astype("int64")
interactions = interactions.sort_values(by="timestamp")
interactions["timestamp"] = interactions.groupby("user_id").cumcount()
interactions

,user_id,item_id,rating,timestamp
1000138,6040,858,4,0
1000153,6040,2384,4,1
999873,6040,593,5,2
1000007,6040,1961,4,3
1000192,6040,2019,5,4
...,...,...,...,...
825793,4958,2399,1,446
825438,4958,1407,5,447
825724,4958,3264,4,448
825731,4958,2634,3,449


### Split interactions into the train, validation and test datasets using LastNSplitter

In [8]:
splitter = LastNSplitter(
    N=1,
    divide_column="user_id",
    query_column="user_id",
    strategy="interactions",
)

raw_test_events, raw_test_gt = splitter.split(interactions)
raw_validation_events, raw_validation_gt = splitter.split(raw_test_events)
raw_train_events = raw_validation_events

### Prepare FeatureSchema required to create Dataset

In [9]:
def prepare_feature_schema(is_ground_truth: bool) -> FeatureSchema:
    base_features = FeatureSchema(
        [
            FeatureInfo(
                column="user_id",
                feature_hint=FeatureHint.QUERY_ID,
                feature_type=FeatureType.CATEGORICAL,
            ),
            FeatureInfo(
                column="item_id",
                feature_hint=FeatureHint.ITEM_ID,
                feature_type=FeatureType.CATEGORICAL,
            ),
        ]
    )
    if is_ground_truth:
        return base_features

    all_features = base_features + FeatureSchema(
        [
            FeatureInfo(
                column="timestamp",
                feature_type=FeatureType.NUMERICAL,
                feature_hint=FeatureHint.TIMESTAMP,
            ),
        ]
    )
    return all_features

### Create Dataset for the training stage

In [10]:
train_dataset = Dataset(
    feature_schema=prepare_feature_schema(is_ground_truth=False),
    interactions=raw_train_events,
    query_features=user_features,
    item_features=item_features,
    check_consistency=True,
    categorical_encoded=False,
)

### Create Datasets (events and ground_truth) for the validation stage

In [11]:
validation_dataset = Dataset(
    feature_schema=prepare_feature_schema(is_ground_truth=False),
    interactions=raw_validation_events,
    query_features=user_features,
    item_features=item_features,
    check_consistency=True,
    categorical_encoded=False,
)
validation_gt = Dataset(
    feature_schema=prepare_feature_schema(is_ground_truth=True),
    interactions=raw_validation_gt,
    check_consistency=True,
    categorical_encoded=False,
)

### Create Datasets (events and ground_truth) for the testing stage

In [12]:
test_dataset = Dataset(
    feature_schema=prepare_feature_schema(is_ground_truth=False),
    interactions=raw_test_events,
    query_features=user_features,
    item_features=item_features,
    check_consistency=True,
    categorical_encoded=False,
)
test_gt = Dataset(
    feature_schema=prepare_feature_schema(is_ground_truth=True),
    interactions=raw_test_gt,
    check_consistency=True,
    categorical_encoded=False,
)

### Create the tensor schema
A schema shows the correspondence of columns from the source dataset with the internal representation of tensors inside the model

In [13]:
ITEM_FEATURE_NAME = "item_id_seq"

tensor_schema = TensorSchema(
    TensorFeatureInfo(
        name=ITEM_FEATURE_NAME,
        is_seq=True,
        feature_type=FeatureType.CATEGORICAL,
        feature_sources=[TensorFeatureSource(FeatureSource.INTERACTIONS, train_dataset.feature_schema.item_id_column)],
        feature_hint=FeatureHint.ITEM_ID,
    )
)

### Create sequential datasets using SequenceTokenizer
The SequentialDataset internally store data in the form of sequences of items sorted by increasing interaction time (timestamp). A SequenceTokenizer is used to convert to this format. In addition, the SequenceTokenizer encodes all categorical columns from the source dataset and stores mapping inside itself.
SequentialDataset.keep_common_query_ids is used to leave only sequences from the same users

In [14]:
tokenizer = SequenceTokenizer(tensor_schema, allow_collect_to_master=True)
tokenizer.fit(train_dataset)

sequential_train_dataset = tokenizer.transform(train_dataset)

sequential_validation_dataset = tokenizer.transform(validation_dataset)
sequential_validation_gt = tokenizer.transform(validation_gt, [tensor_schema.item_id_feature_name])

sequential_validation_dataset, sequential_validation_gt = SequentialDataset.keep_common_query_ids(
    sequential_validation_dataset, sequential_validation_gt
)

In [15]:
test_query_ids = test_gt.query_ids
test_query_ids_np = tokenizer.query_id_encoder.transform(test_query_ids)["user_id"].values
sequential_test_dataset = tokenizer.transform(test_dataset).filter_by_query_id(test_query_ids_np)

## Train model
### Create SASRec model instance and run the training stage using lightning
After each epoch validation metrics are shown. You can change the list of validation metrics in ValidationMetricsCallback
The model is determined to be the best and is saved if the metric updates its maximum during validation (see the ModelCheckpoint)

In [ ]:
MAX_SEQ_LEN = 200
BATCH_SIZE = 512
NUM_WORKERS = 4
MAX_EPOCHS = 1

model = SasRec(
    tensor_schema,
    block_count=2,
    head_count=2,
    max_seq_len=MAX_SEQ_LEN,
    hidden_size=300,
    dropout_rate=0.5,
    optimizer_factory=FatOptimizerFactory(learning_rate=0.001),
)

csv_logger = CSVLogger(save_dir=".logs/train", name="SASRec_example")

checkpoint_callback = ModelCheckpoint(
    dirpath=".checkpoints",
    save_top_k=1,
    verbose=True,
    # if you use multiple dataloaders, then add the serial number of the dataloader to the suffix of the metric name.
    # For example,"recall@10/dataloader_idx_0"
    monitor="recall@10",
    mode="max",
)

validation_metrics_callback = ValidationMetricsCallback(
    metrics=["map", "ndcg", "recall"],
    ks=[1, 5, 10, 20],
    item_count=train_dataset.item_count,
    postprocessors=[RemoveSeenItems(sequential_validation_dataset)],
)

trainer = L.Trainer(
    max_epochs=MAX_EPOCHS,
    callbacks=[checkpoint_callback, validation_metrics_callback],
    logger=csv_logger,
)

train_dataloader = DataLoader(
    dataset=SasRecTrainingDataset(
        sequential_train_dataset,
        max_sequence_length=MAX_SEQ_LEN,
    ),
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=NUM_WORKERS,
    pin_memory=True,
)

validation_dataloader = DataLoader(
    dataset=SasRecValidationDataset(
        sequential_validation_dataset,
        sequential_validation_gt,
        sequential_train_dataset,
        max_sequence_length=MAX_SEQ_LEN,
    ),
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    pin_memory=True,
)

trainer.fit(
    model,
    train_dataloaders=train_dataloader,
    val_dataloaders=validation_dataloader,
)

The path to the best model is saved inside checkpoint_callback

In [18]:
best_model = SasRec.load_from_checkpoint(checkpoint_callback.best_model_path).eval()

## Inference stage
### Prepare Dataloader and logger

In [19]:
prediction_dataloader = DataLoader(
    dataset=SasRecPredictionDataset(
        sequential_test_dataset,
        max_sequence_length=MAX_SEQ_LEN,
    ),
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    pin_memory=True,
)

csv_logger = CSVLogger(save_dir=".logs/test", name="SASRec_example")

### Run inference
You can get the recommendations in four formats: PySpark DataFrame, Pandas DataFrame, Polars DataFrame, PyTorch tensors. Each of the types corresponds a callback.

You can filter the results using postprocessors strategy. For example the RemoveSeenItems postprocessor is filtering out the items that already have been seen in test dataset.

You don't need to use all three callbacks. This is shown only for example

Also, you can get user embeddings, that were used to perform predictions, using `get_query_embedding` method inside SasRecModel or `QueryEmbeddingsPredictionCallback` for lightning module.

To operate with PySpark DataFrames and use ``SparkPredictionCallback`` you should create a spark session.

In [ ]:
from replay.utils.session_handler import get_spark_session
spark_session = get_spark_session()

In [ ]:
TOPK = [1, 10, 20, 100]

postprocessors = [RemoveSeenItems(sequential_test_dataset)]

spark_prediction_callback = SparkPredictionCallback(
    spark_session=spark_session,
    top_k=max(TOPK),
    query_column="user_id",
    item_column="item_id",
    rating_column="score",
    postprocessors=postprocessors,
)

pandas_prediction_callback = PandasPredictionCallback(
    top_k=max(TOPK),
    query_column="user_id",
    item_column="item_id",
    rating_column="score",
    postprocessors=postprocessors,
)

torch_prediction_callback = TorchPredictionCallback(
    top_k=max(TOPK),
    postprocessors=postprocessors,
)

query_embeddings_callback = QueryEmbeddingsPredictionCallback()

trainer = L.Trainer(
    callbacks=[
        spark_prediction_callback,
        pandas_prediction_callback,
        torch_prediction_callback,
        query_embeddings_callback,
    ],
    logger=csv_logger,
    inference_mode=True,
)
trainer.predict(best_model, dataloaders=prediction_dataloader, return_predictions=False)

spark_res = spark_prediction_callback.get_result()
pandas_res = pandas_prediction_callback.get_result()
torch_user_ids, torch_item_ids, torch_scores = torch_prediction_callback.get_result()
user_embeddings = query_embeddings_callback.get_result()

In [23]:
spark_res.show()

+-------+-------+------------------+
|user_id|item_id|             score|
+-------+-------+------------------+
|      0|   1178|3.5885117053985596|
|      0|    476|3.5384256839752197|
|      0|   2789| 3.507075548171997|
|      0|    585| 3.505156993865967|
|      0|   1192| 3.410585403442383|
|      0|   1539| 3.384791851043701|
|      0|   1575|3.3331282138824463|
|      0|   2502|  3.29717755317688|
|      0|   1196|3.2777607440948486|
|      0|    108|3.2731778621673584|
|      0|   1180|3.2617220878601074|
|      0|   1220| 3.234489679336548|
|      0|   2559| 3.188918352127075|
|      0|    847|3.1777091026306152|
|      0|   2847|3.1535818576812744|
|      0|   3509|3.1391689777374268|
|      0|     49|3.1181435585021973|
|      0|    770| 3.102733612060547|
|      0|    352| 3.085745334625244|
|      0|   2928|3.0841736793518066|
+-------+-------+------------------+
only showing top 20 rows



In [24]:
pandas_res

,user_id,item_id,score
0,0,1178,3.588512
0,0,476,3.538426
0,0,2789,3.507076
0,0,585,3.505157
0,0,1192,3.410585
...,...,...,...
6039,6039,3012,2.21179
6039,6039,2199,2.203226
6039,6039,3441,2.192126
6039,6039,1468,2.188128


In [25]:
print(torch_user_ids[0], torch_item_ids[0], torch_scores[0])

tensor(0) tensor([1178,  476, 2789,  585, 1192, 1539, 1575, 2502, 1196,  108, 1180, 1220,
        2559,  847, 2847, 3509,   49,  770,  352, 2928,  537, 1023, 3106,  453,
         315, 1182,   33, 3724, 1120,  293, 1931, 1203, 2647, 1245,  912,  642,
        1366,  373, 1271, 2327, 1656, 2530,   31, 1533, 2614, 2637,  220,  589,
        1195, 2918, 2105, 1353, 2222, 2890, 1726,  900, 1373, 3107, 1287, 3682,
        1491,  376,  724, 3412, 3684, 1899, 1372, 1284, 3078, 1207, 1854, 1073,
        1188,  588,  740,   38, 1239, 1568, 1111, 1201, 1063, 2630, 2631, 1211,
        1505, 1595, 1543,   20, 1227, 2255, 1268, 3402, 1258, 3349,  911,  583,
        1202, 2916, 1058, 2641]) tensor([3.5885, 3.5384, 3.5071, 3.5052, 3.4106, 3.3848, 3.3331, 3.2972, 3.2778,
        3.2732, 3.2617, 3.2345, 3.1889, 3.1777, 3.1536, 3.1392, 3.1181, 3.1027,
        3.0857, 3.0842, 3.0725, 3.0314, 3.0309, 3.0186, 3.0083, 2.9766, 2.9718,
        2.9429, 2.9349, 2.9295, 2.9257, 2.9225, 2.9016, 2.8875, 2.8823, 2.875

Suppose we want to get the recomendations in PySpark format. 
Let's get the inverse representation of labels using inverse_transform method.

Note that the reverse representation can only be obtained for PySpark and Pandas formats. When working with PyTorch tensors, the reverse representation must be done manually

In [26]:
recommendations = tokenizer.query_and_item_id_encoder.inverse_transform(spark_res)

In [27]:
recommendations.show()

+------------------+-------+-------+
|             score|user_id|item_id|
+------------------+-------+-------+
|3.5885117053985596|      1|   1196|
|3.5384256839752197|      1|    480|
| 3.507075548171997|      1|   2858|
| 3.505156993865967|      1|    589|
| 3.410585403442383|      1|   1210|
| 3.384791851043701|      1|   1580|
|3.3331282138824463|      1|   1617|
|  3.29717755317688|      1|   2571|
|3.2777607440948486|      1|   1214|
|3.2731778621673584|      1|    110|
|3.2617220878601074|      1|   1198|
| 3.234489679336548|      1|   1240|
| 3.188918352127075|      1|   2628|
|3.1777091026306152|      1|    858|
|3.1535818576812744|      1|   2916|
|3.1391689777374268|      1|   3578|
|3.1181435585021973|      1|     50|
| 3.102733612060547|      1|    780|
| 3.085745334625244|      1|    356|
|3.0841736793518066|      1|   2997|
+------------------+-------+-------+
only showing top 20 rows



#### Run inference on a subset of items
It happens that it is necessary to process an inference not on all items, but on a certain subset (we will call it ``candidates``). For example, you want to make predictions only for the cartoons among all possible movies. 

To speed up the inference in this case, you can use the SasRec's property ``candidates_to_score``. It should be a ``torch.LongTensor`` with the IDs of the objects on which you want to process an inference. It is important that the candidate scores will be returned in the order in which their IDs were in the ``candidates_to_score``.

In [28]:
best_model_candidates = SasRec.load_from_checkpoint(checkpoint_callback.best_model_path)

In [ ]:
TOPK = 2
CANDIDATES = torch.LongTensor([42, 1337])

postprocessors = [RemoveSeenItems(sequential_test_dataset)]

pandas_prediction_callback = PandasPredictionCallback(
    top_k=TOPK,
    query_column="user_id",
    item_column="item_id",
    rating_column="score",
    postprocessors=postprocessors,
)

trainer = L.Trainer(callbacks=[pandas_prediction_callback], logger=csv_logger, inference_mode=True)
best_model_candidates.candidates_to_score = CANDIDATES
trainer.predict(best_model_candidates, dataloaders=prediction_dataloader, return_predictions=False)

There will be scores only for items whose IDs are contained in ``candidates_to_score``.

If ``candidates_to_score`` contains a small number of candidates and ``top_k`` parameter is small, it may happen that the required number of items will not remain after the postprocessor is running. In this case, the ``top_k`` items for each user will be returned from the model, then the postprocessor will remove the seen items and if the user has less than the ``top_k`` items, then the non-candidate items with a score equal to ``-inf`` will be added to it.

In [33]:
pandas_prediction_callback.get_result()

,user_id,item_id,score
0,0,1337,2.424251
0,0,42,-0.840065
1,1,1337,2.412392
1,1,42,-0.81834
2,2,1337,2.419046
...,...,...,...
6037,6037,42,-0.809542
6038,6038,1337,2.415145
6038,6038,42,-0.826277
6039,6039,42,-0.799789


**Note:** don`t forget to reset ``candidates_to_score`` to ``None`` if they are no longer needed and you want to run the model inference with all items.

In [ ]:
best_model_candidates.candidates_to_score = None

### Calculating metrics

In [34]:
init_args = {"query_column": "user_id", "rating_column": "score"}

In [35]:
result_metrics = OfflineMetrics(
    [Recall(TOPK), Precision(TOPK), MAP(TOPK), NDCG(TOPK), MRR(TOPK), HitRate(TOPK)], **init_args
)(recommendations.toPandas(), raw_test_gt)

In [36]:
metrics_to_df(result_metrics)

k,2
HitRate,0.008278
MAP,0.006291
MRR,0.006291
NDCG,0.006812
Precision,0.004139
Recall,0.008278


### User embeddings

Got 6040 x 300 user embeddings, because among all 12 batches: 

11 batches contains 512 samples

1 batch contains 408 left samples

11 * 512 + 408 == 6040

In [37]:
user_embeddings

tensor([[-1.0552,  0.4752,  1.0247,  ...,  1.1565, -1.0123,  0.8807],
        [-1.0668,  0.4451,  1.0222,  ...,  1.1616, -0.9487,  0.8842],
        [-0.9756,  0.4984,  1.0899,  ...,  1.1721, -1.0131,  0.9048],
        ...,
        [-0.9972,  0.4614,  0.9711,  ...,  1.1111, -0.9304,  0.9788],
        [-1.0409,  0.4961,  1.0886,  ...,  1.1576, -0.9697,  0.9456],
        [-1.0410,  0.4870,  1.0076,  ...,  1.1648, -0.9412,  0.9579]])

In [38]:
user_embeddings.shape

torch.Size([6040, 300])

You can access user embeddings directly with `SasRecModel` class

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

core_model = SasRecModel(tensor_schema, num_blocks=2, num_heads=2, max_len=MAX_SEQ_LEN, hidden_size=300, dropout=0.5)
core_model.eval()
core_model = core_model.to(device)

# Get first batch of data
data = next(iter(prediction_dataloader))
tensor_map, padding_mask = data["feature_tensor"], data["padding_mask"]

# Ensure everything is on the same device
padding_mask = padding_mask.to(device)
tensor_map["item_id_seq"] = tensor_map["item_id_seq"].to(device)

# Get user embeddings
user_embeddings_batch = core_model.get_query_embeddings(tensor_map, padding_mask)
user_embeddings_batch

tensor([[ 0.9250, -0.2808,  0.4892,  ..., -0.2213, -0.2618, -1.6309],
        [ 0.6379, -0.6346,  0.6803,  ..., -0.8519, -1.8054, -0.2082],
        [ 0.6610, -1.2338, -1.1149,  ..., -1.1833,  0.1578, -1.6709],
        ...,
        [ 0.8832,  0.4134,  1.1514,  ...,  1.6798,  1.2664, -0.1323],
        [ 0.7696, -1.0891, -0.0255,  ..., -0.6802, -2.3859,  0.9376],
        [ 0.3594,  0.4212, -0.1415,  ..., -1.7924,  0.8588,  1.4970]],
       grad_fn=<SliceBackward0>)

In [40]:
user_embeddings_batch.shape

torch.Size([512, 300])

### Item embeddings

`get_all_embeddings()` method in transformers can be used to get copies of all embeddings that are presented in model as a dict.

In [41]:
all_embeddings = best_model.get_all_embeddings()
all_embeddings

{'item_embedding': tensor([[-0.0088,  0.0053,  0.0240,  ...,  0.0405, -0.0129, -0.0295],
         [-0.0342, -0.0158,  0.0267,  ..., -0.0280,  0.0091, -0.0065],
         [-0.0102,  0.0370,  0.0512,  ...,  0.0010, -0.0135,  0.0495],
         ...,
         [ 0.0089, -0.0347, -0.0122,  ...,  0.0173, -0.0082, -0.0398],
         [-0.0394, -0.0049, -0.0017,  ...,  0.0211,  0.0258, -0.0514],
         [-0.0051, -0.0094,  0.0008,  ..., -0.0190,  0.0255,  0.0364]]),
 'positional_embedding': tensor([[ 0.1177, -0.0036, -0.0918,  ..., -0.1263, -0.0596, -0.0295],
         [-0.0880, -0.0568,  0.0890,  ...,  0.1338, -0.0961, -0.0389],
         [ 0.1122,  0.0297, -0.0375,  ...,  0.0995, -0.0452, -0.0724],
         ...,
         [-0.0280,  0.0620, -0.0093,  ...,  0.0499,  0.1338, -0.0431],
         [-0.0352,  0.0070,  0.0676,  ...,  0.1163,  0.0015, -0.0117],
         [ 0.0129,  0.0570, -0.0643,  ...,  0.1092, -0.0102,  0.0391]])}

You can access item embeddings from this dictionary

In [42]:
item_embeddings = all_embeddings["item_embedding"]
item_embeddings

tensor([[-0.0088,  0.0053,  0.0240,  ...,  0.0405, -0.0129, -0.0295],
        [-0.0342, -0.0158,  0.0267,  ..., -0.0280,  0.0091, -0.0065],
        [-0.0102,  0.0370,  0.0512,  ...,  0.0010, -0.0135,  0.0495],
        ...,
        [ 0.0089, -0.0347, -0.0122,  ...,  0.0173, -0.0082, -0.0398],
        [-0.0394, -0.0049, -0.0017,  ...,  0.0211,  0.0258, -0.0514],
        [-0.0051, -0.0094,  0.0008,  ..., -0.0190,  0.0255,  0.0364]])

Item embeddings shape is (N_ITEMS, HIDDEN_SIZE)

In [43]:
item_embeddings.shape

torch.Size([3883, 300])

Ensure we got correct dimension and ensure we got the copy of tensor

In [44]:
assert item_embeddings.shape[0] == len(tokenizer.item_id_encoder.mapping["item_id"])
assert id(item_embeddings) != id(best_model._model.item_embedder.item_emb.weight.data)

For example we observe one new item id in our training data. We can easily expand our item embedder by one element

In order to expand item embeddings by new size `set_item_embeddings_by_size` method is applied

In [45]:
best_model.set_item_embeddings_by_size(item_embeddings.shape[0] + 1)

Now our new item embeddings have one extra embedding

In [46]:
new_size = best_model.get_all_embeddings()["item_embedding"].shape[0]
old_size = item_embeddings.shape[0]

assert new_size == old_size + 1

Alternatively, we can pass our item embeddings that replace the existing ones by calling `set_item_embeddings_by_tensor`.

If tensor contains new items, they will be added to item embedder.

In [47]:
new_embeddings_weights = torch.rand((new_size + 1, 300))  # randint used for example only

best_model.set_item_embeddings_by_tensor(new_embeddings_weights)

At the moment we expanded our item embeddings by one more item and replace weights by passing `new_embeddings_weights`

In [48]:
old_size = new_size
new_size = best_model.get_all_embeddings()["item_embedding"].shape[0]

assert new_size == old_size + 1

Similarly, we can append tensor for only new items with no replace for existing by calling `append_item_embeddings`

In [49]:
new_item_weights = torch.rand((1, 300))  # randint used for example only

best_model.append_item_embeddings(new_item_weights)

We passed one new example and its weights to item embeddings, thus expanded our vocabulary by one item again

In [50]:
old_size = new_size
new_size = best_model.get_all_embeddings()["item_embedding"].shape[0]

assert new_size == old_size + 1

## Example of launching an inference for a single user without using a trainer (in order to speed up)
An example for the production of an online script

Let's assume that the user's sequence consisted of a sequence of items [1, 2, 3, 4, 5]. 
Сreate a padding mask corresponding to the sequence of items.

It is important to take only the latest MAX_SEQ_LEN or less items.

You can use ``candidates_to_score`` here as well. It is possible to set a property or pass ``candidates_to_score`` as a parameter of ``predict`` method.

**Note**: make sure that you set the ``torch.set_num_threads()`` parameter in the product environment. This is important because torch can consume resources exceeding the k8s limit and thus activating CPU throttling.

#### Create sequence and padding_mask

In [51]:
item_sequence = torch.arange(1, 5).unsqueeze(0)[:, -MAX_SEQ_LEN:]
padding_mask = torch.ones_like(item_sequence, dtype=torch.bool)
sequence_item_count = item_sequence.shape[1]

#### Wrapping created tensors in the SasRecPredictionBatch entity

In [78]:
batch = SasRecPredictionBatch(
    query_id=torch.arange(0, item_sequence.shape[0], 1).long(),
    padding_mask=padding_mask.bool(),
    features={ITEM_FEATURE_NAME: item_sequence.long()},
)

### Run predict step of the SasRec and get scores from the model

In [69]:
with torch.no_grad():
    scores = best_model.predict(batch)
scores

tensor([[-5.6152,  2.8685,  3.3463,  ...,  4.3111,  2.4290, -7.1570]])

You can pass ``candidates_to_score`` in ``predict()``.

In [72]:
with torch.no_grad():
    scores = best_model.predict(batch, candidates_to_score=CANDIDATES)
scores

tensor([[-4.7949, -4.2651]])

### Getting three items with the highest score

In [76]:
with torch.no_grad():
    scores = best_model.predict(batch)
torch.topk(scores, k=3).indices

tensor([[ 863, 1828, 1170]])

### Optimized inference on CPU with OpenVino
SasRec model can be compiled into IR format of OpenVino for faster inference on CPU.

SasRec model itself or the path to the checkpoint of the model can be passed as ``model`` parameter .

Parameter ``mode`` defines inference mode and shape of inputs. Could be one of ``one_query``, ``batch``, ``dynamic_batch_size``. This parameter determines whether the first dimension of the input (the batch size) will be static or dynamic.

Parameter ``num_candidates_to_score`` defines number of item ids to calculate scores if it is necessary. This parameter determines whether the model will make a partial inference and, if so, whether the list of candidates will have a static or dynamic length.

Parameter ``num_threads`` defines number of CPU threads to use.

In [ ]:
from replay.models.nn.sequential.compiled import SasRecCompiled

In [80]:
best_model = SasRec.load_from_checkpoint(checkpoint_callback.best_model_path)

Compile model from SasRec model or checkpoint.

In [81]:
opt_model = SasRecCompiled.compile(
    model=best_model,  # or checkpoint_callback.best_model_path
    mode="one_query",
)

Wrapping tensors in the SasRecPredictionBatch entity

In [82]:
item_sequence = torch.arange(1, 5).unsqueeze(0)[:, -MAX_SEQ_LEN:]
padding_mask = torch.ones_like(item_sequence, dtype=torch.bool)
sequence_item_count = item_sequence.shape[1]

In [83]:
batch = SasRecPredictionBatch(
    query_id=torch.arange(0, item_sequence.shape[0], 1).long(),
    padding_mask=padding_mask.bool(),
    features={ITEM_FEATURE_NAME: item_sequence.long()},
)

Run predict and get scores from the model

In [84]:
opt_model.predict(batch).shape

torch.Size([1, 3883])

#### Compiled model also supports inference on submitted candidates.

Wrapping created tensors in the SasRecPredictionBatch entity

In [93]:
batch = SasRecPredictionBatch(
    query_id=torch.arange(0, item_sequence.shape[0], 1).long(),
    padding_mask=padding_mask.bool(),
    features={ITEM_FEATURE_NAME: item_sequence.long()},
)

Compile model with defined ``num_candidates_to_score``. There are 3 alternatives:
- ``-1`` - sets ``candidates_to_score`` shape to dynamic range [1, ?]
- ``N`` - sets ``candidates_to_score`` shape to [1, N]
- ``None`` - disable ``candidates_to_score`` usage

In [94]:
opt_model = SasRecCompiled.compile(
    model=best_model,
    mode="one_query",
    num_candidates_to_score=2,
)

Run predict and get scores from the model

In [95]:
opt_model.predict(batch, CANDIDATES).shape

torch.Size([1, 2])